In [3]:
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/03/e6/4aef6799badc2693548559bad5b56d56cfe89eada337c815fdfe92175250/xgboost-2.0.3-py3-none-macosx_12_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 5.4 MB/s eta 0:00:00a 0:00:01


In [29]:
import numpy as np
import pandas as pd
 
import xgboost as xgb
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import os
import glob

In [5]:
# Import expected points csv by loading into DF?
#TODO: this
# Example DataFrame
data = {
    'feature1': [1, 2, 3, 4, 5],
    'feature2': [5, 4, 3, 2, 1],
    'target': [1, 2, 3, 4, 5]
}
df = pd.DataFrame(data)

In [15]:
def combine_csv_files(root_dir, target_filename):
    """
    Traverse through folders in the root_dir, find all CSV files with the name target_filename,
    and combine them into one pandas DataFrame.

    Args:
    root_dir (str): The root directory to start the traversal.
    target_filename (str): The name of the CSV files to search for.

    Returns:
    pandas.DataFrame: A DataFrame containing the combined data from all found CSV files.
    """
    # Create an empty DataFrame to hold combined data
    combined_df = pd.DataFrame()

    # Walk through all directories and subdirectories starting from root_dir
    for subdir, dirs, files in os.walk(root_dir):
        # Construct the full path to target CSV files within this directory
        file_pattern = os.path.join(subdir, target_filename)
        # Use glob to find all files matching the pattern
        for file in glob.glob(file_pattern):
            # Read the CSV file into a DataFrame
            df = pd.read_csv(file, header=1)
            # Append the data of this CSV file to the combined DataFrame
            combined_df = pd.concat([combined_df, df], ignore_index=True)

    return combined_df

# Example usage
root_directory = '/Users/rob/Desktop/ECs/AoSB/raw_data/2023'  # Change this to your root directory
target_csv_name = 'expected_points.csv'     # Change this to your target CSV filename
end_dataframe = combine_csv_files(root_directory, target_csv_name)

# # Optional: Save the combined DataFrame to a new CSV file
# resulting_dataframe.to_csv('/path/to/save/combined_csv.csv', index=False)
df = end_dataframe.drop(['Tm'], axis=1)




,Total,Tot,Pass,Rush,TOvr,Tot.1,Pass.1,Rush.1,TOvr.1,Tot.2,KO,KR,P,PR,FG/XP
0,14.0,10.64,16.40,-6.90,-7.87,9.73,8.46,0.94,6.96,-3.59,-3.53,2.31,2.02,-3.85,-0.54
1,-14.0,-9.73,-8.46,-0.94,-6.96,-10.64,-16.40,6.90,7.87,3.59,-2.31,3.53,3.85,-2.02,0.54
2,9.0,-2.59,4.97,-7.24,-15.00,3.68,4.06,0.35,-0.77,2.01,-4.12,2.75,0.26,-1.49,4.61
3,-9.0,-3.68,-4.06,-0.35,0.77,2.59,-4.97,7.24,15.00,-2.01,-2.75,4.12,1.49,-0.26,-4.61
4,-9.0,-3.72,-2.08,-2.88,-7.32,-9.20,-2.73,-6.80,0.00,1.65,-1.96,3.20,2.88,-3.97,1.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,-1.0,-8.89,-8.57,-0.32,-11.26,3.31,5.82,-2.51,0.61,1.81,-2.50,3.69,2.02,-2.44,1.04
566,7.0,-7.10,-1.27,-2.58,-6.83,12.05,3.80,7.92,8.65,3.24,-0.18,4.80,1.87,-2.53,-0.72
567,-7.0,-12.05,-3.80,-7.92,-8.65,7.10,1.27,2.58,6.83,-3.24,-4.80,0.18,2.53,-1.87,0.72
568,0.0,11.30,14.09,-2.79,-3.79,-9.48,-4.96,-4.52,3.77,2.42,-2.44,2.44,0.81,-1.32,2.93


In [48]:
# Define features and target
X = df.drop(['Total'], axis=1)  # or df.drop('target', axis=1) to use all other columns as features
y = df['Total']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

In [49]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

In [56]:
# The coefficients
# print(list(X.columns))
print("Coefficients: \n")
for a, b in list(zip(list(X.columns), regr.coef_)):
    print(a+": "+str(b))

print('\n')
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))


Coefficients: 

Tot: 1.1003744248263572
Pass: -0.12594511926127777
Rush: -0.13779808464108959
TOvr: -0.02069178284225751
Tot.1: 1.0511888763778559
Pass.1: -0.0864904361488988
Rush.1: -0.09702339430043391
TOvr.1: -0.013929061894789475
Tot.2: 0.7584782032016216
KO: 0.09277233476713953
KR: 0.0712821361227717
P: 0.2051842682914233
PR: 0.20787942501668416
FG/XP: 0.1813600390036012


Mean squared error: 4.87
Coefficient of determination: 0.98


In [60]:
# Instantiate an XGBRegressor
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3)

# Train the model
model.fit(X_train, y_train)

# model.coef_

# Make predictions
y_pred = model.predict(X_test)

In [59]:
# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"r2 Score: {r2}")

Mean Squared Error: 4.868242471205374
r2 Score: 0.9765042682038773
